In [ ]:
pip install chaospy

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#Escreva seu nome e numero USP
INFO = {11811012:"Vinícius da Costa Collaço"}
A = 0.460279  # A = 0.rg (6 dígitos siginificativos)
B = 0.382023 # B = 0.cpf (6 dígitos siginificativos)

In [ ]:
#Bibliotecas
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time
import chaospy as cp


In [ ]:
def gera_imagem():
  """
  Funcao para gera imagem comparativa no relatorio
  """

  uniform_cube = cp.J(cp.Uniform(0, 1),cp.Uniform(0,1))
  exp_cube = cp.J(cp.TruncExponential(upper=1,scale=0.460279),cp.TruncExponential(upper=1,scale=0.460279))
  count = 400

  random_samples = uniform_cube.sample(count, rule="random")
  sobol_samples = uniform_cube.sample(count, rule="sobol")
  random_samples_te = exp_cube.sample(count, rule="random")
  sobol_samples_te = exp_cube.sample(count, rule="sobol")


  plt.rc("figure", figsize=[14, 12])

  plt.subplot(221)
  plt.scatter(*random_samples, s=10, c='g')
  plt.title("Distribuição uniforme pseudo aleatória")


  plt.subplot(222)
  plt.scatter(*sobol_samples, s=10, c='r')
  plt.title("Distribuição uniforme quase aleatória (Sobol)")

  plt.subplot(223)
  plt.scatter(*random_samples_te, s=10, c='g')
  plt.title("Distribuição exponencial pseudo aleatória")

  plt.subplot(224)
  plt.scatter(*sobol_samples_te, s=10, c='r')
  plt.title("Distribuição exponencial quase aleatória (Sobol)")

  plt.savefig('Comparacao_Distribuicao')

In [ ]:
def gera_ponto(n):
  """
  Funcao para gerar um array com n pontos quase aleatorios
  """
  dist_uniforme = cp.Uniform(0, 1)
  x = dist_uniforme.sample(n, rule = "sobol")

  return x

In [ ]:
def gera_pontos(n):
  """
  Funcao para gerar dois array com n pares de pontos quase aleatorios
  """

  quadrado_uniforme = cp.J(cp.Uniform(0, 1), cp.Uniform(0, 1))
  x = quadrado_uniforme.sample(n, rule = "sobol")

  return x[0], x[1]

In [ ]:
def gera_ponto_exp(n):
  """
  Funcao para gerar um array com n pontos quase aleatorios
  com distribuicao exponencial truncada
  """
  b = 0.460279
  dist_exp = cp.TruncExponential(upper=1,scale=1/b)
  x = dist_exp.sample(n, rule = "sobol")

  return x


In [ ]:
def indicadora (vetor):
  """
  Indicadora da Hit or Miss (1 ou 0)
  """
  true_or_false = vetor[1] <= f(vetor[0])

  return np.where(true_or_false == True,1,0)

In [ ]:
def f_sobre_g(x):
  """
  Funcao agregada de f(x)/g(x) da importance sampling
  """
  A = 0.460279  # A = 0.rg (6 dígitos siginificativos)
  B = 0.382023 # B = 0.cpf (6 dígitos siginificativos)
  f_sobre_g = (np.cos(B*x)*(1 - np.exp(-A)))/A

  return f_sobre_g


In [ ]:
def phi(x):
  """
  Funcao phi(x) controle do metodo control variate
  """
  phi = (np.array(-x)/2.5) + 1
  
  return phi

In [ ]:
def f(x):
  """
  Esta funcao deve receber x e devolver f(x), como especifcado no enunciado
  Escreva o seu codigo nas proximas linhas
  """

  A = 0.460279  # A = 0.rg (6 dígitos siginificativos)
  B = 0.382023 # B = 0.cpf (6 dígitos siginificativos)
  F = (np.exp(-A*x))*np.cos(B*x)
  
  return F

In [ ]:
def crude(Seed=None, n=352469):
    """
    Esta funcao deve retornar a sua estimativa para o valor da integral de f(x)
    usando o metodo crude
    Escreva o seu codigo nas proximas linhas
    """
    random.seed(Seed)
    np.random.seed(Seed)

    pontos = gera_ponto (n)
    array_estimador = f(pontos)
    estima_crude = np.sum(array_estimador)/n

    return estima_crude

In [ ]:
def hit_or_miss(Seed=None, n=4334113):
    """
    Esta funcao deve retornar a sua estimativa para o valor da integral de f(x)
    usando o metodo hit or miss
    Escreva o seu codigo nas proximas linhas
    """
    random.seed(Seed)
    np.random.seed(Seed)
    
    pontos = gera_pontos (n)
    pontos_dentro = indicadora (pontos)
    estima_HoM = sum(pontos_dentro)/n

    return estima_HoM

In [ ]:
def importance_sampling(Seed=None, n=6781):
    
    """
    Esta funcao deve retornar a sua estimativa para o valor da integral de f(x)
    usando o metodo importance sampling
    Escreva o seu codigo nas proximas linhas
    """
    random.seed(Seed)
    np.random.seed(Seed)

    pontos = gera_ponto_exp(n)
    array_estimador = f_sobre_g(pontos)
    estima_importance_sampling = np.sum(array_estimador)/n

    return estima_importance_sampling


In [ ]:
def control_variate(Seed=None, n=842):
    """
    Esta funcao deve retornar a sua estimativa para o valor da integral de f(x)
    usando o metodo control variate
    Escreva o seu codigo nas proximas linhas
    """
    random.seed(Seed)
    np.random.seed(Seed)

    pontos = gera_ponto(n)
    array_f = f(pontos)
    array_phi = phi(pontos)
    integral_phi = 0.8 #calculada algebricamente
    estima_control_variate = (np.mean(array_f) 
                              - np.mean(array_phi) 
                              + integral_phi)
    
    return estima_control_variate

In [ ]:
def variancias (Seed=38, n=100):
  """ 
  Funcao para calculo das variancias empiricas
  """  
  random.seed(Seed)
  np.random.seed(Seed)

  #Variancia do metodo Crude
  pontos = gera_ponto (n)
  array_estimador = f(pontos)
  var_crude = np.var(array_estimador, ddof=1)

  #Variancia do metodo Hit or Miss
  estima_HoM = hit_or_miss(Seed=Seed,n=n)
  var_HoM = (estima_HoM * (1 - estima_HoM))

  #Variancia do metodo Importance Sampling
  
  pontos = gera_ponto_exp(n)
  array_estimador_IS = f_sobre_g(pontos)
  var_IS = np.var(array_estimador_IS, ddof=1)

  #Variancia do metodo Control Variate
  pontos = gera_ponto(n)
  array_f = f(pontos)
  array_phi = phi(pontos)
  var_CV = (np.var(array_f, ddof=1) 
            + np.var(array_phi, ddof=1) 
            - 2*np.cov(array_f,array_phi,ddof=1)[0,1])
  
  return var_crude, var_HoM, var_IS, var_CV

In [ ]:
def main():
    #Coloque seus testes aqui
    z = 1.96
    var_crude, var_hom, var_IS, var_CV = variancias()

    crude_est = crude(Seed=38, n=100)
    Ec = 0.0005 * crude_est
    n_crude = math.ceil(var_crude * z**2/Ec**2)
    t0 = time.time()
    MCc = crude(Seed=38, n=n_crude)
    t1 = time.time()
    print('Método Crude = ',MCc)
    print('Erro Crude =', Ec)
    print('Variancia Crude = ',var_crude)
    print('n_crude = ',n_crude)
    print('Tempo Crude = ',t1 - t0, 'segundos')
    print('Eficiência Crude = ', 1/((t1 - t0)*var_crude))

    hom_est = hit_or_miss(Seed=38, n=100)
    Ehom = 0.0005 * hom_est
    nhom = math.ceil(var_hom * z**2/Ehom**2)
    t0 = time.time()
    MChom = hit_or_miss(Seed=38, n=nhom)
    t1 = time.time()
    print('\nMétodo Hit or Miss = ',MChom)
    print('Erro Hit or Miss =', Ehom)
    print('Variancia Hit or Miss = ', var_hom)
    print('n_hom = ',nhom)
    print ('Tempo Hit or Miss = ',t1 - t0, 'segundos')
    print('Eficiência Hit or Miss = ', 1/((t1 - t0)*var_hom))


    IS_est = importance_sampling(Seed=38, n=100)
    EIS = 0.0005 * IS_est
    nIS = math.ceil(var_IS * z**2/EIS**2)
    t0 = time.time()
    MCIS = importance_sampling(Seed=38, n=nIS)
    t1 = time.time() 
    print('\nMétodo Importance Sampling = ',MCIS)
    print('Erro Importance Sampling =', EIS)
    print('Variancia Importance Sampling = ', var_IS)
    print('n_IS = ',nIS)
    print('Tempo Importance Sampling = ',t1 - t0, 'segundos')
    print('Eficiência Importance Sampling = ', 1/((t1 - t0)*var_IS))

    CV_est = control_variate(Seed=38, n=100)
    ECV = 0.0005 * CV_est
    nCV = math.ceil(var_CV * z**2/ECV**2)
    t0 = time.time()
    MCCV = control_variate(Seed=38, n=nCV)
    t1 = time.time() 
    print('\nMétodo Control Variate = ',MCCV)
    print('Erro Control Variate =', ECV)
    print('Variancia Control Variate = ', var_CV)
    print('n_CV = ',nCV)
    print('Tempo Control Variate = ',t1 - t0, 'segundos')
    print('Eficiência Control Variate = ', 1/((t1 - t0)*var_CV))

    #Erros
    real = 0.78428
    print('\nErros')
    print(abs(crude()-real)/real)
    print(abs(hit_or_miss()-real)/real)
    print(abs(importance_sampling()-real)/real)
    print(abs(control_variate()-real)/real)

if __name__ == "___main__":
    main()

In [ ]:
main()

Método Crude =  0.7842812135628642
Erro Crude = 0.00039245464965070826
Variancia Crude =  0.014131472991153545
n_crude =  352469
Tempo Crude =  1.729335069656372 segundos
Eficiência Crude =  40.91979209213054

Método Hit or Miss =  0.7842861965066439
Erro Hit or Miss = 0.00039000000000000005
Variancia Hit or Miss =  0.17159999999999997
n_hom =  4334113
Tempo Hit or Miss =  21.48796820640564 segundos
Eficiência Hit or Miss =  0.27119855034821894

Método Importance Sampling =  0.7842799650312312
Erro Importance Sampling = 0.0003922702094961755
Variancia Importance Sampling =  0.00027160931634469525
n_IS =  6781
Tempo Importance Sampling =  0.03398895263671875 segundos
Eficiência Importance Sampling =  108322.22086712052

Método Control Variate =  0.7842739081121615
Erro Control Variate = 0.0003920952746507083
Variancia Control Variate =  3.3664518880260896e-05
n_CV =  842
Tempo Control Variate =  0.009617090225219727 segundos
Eficiência Control Variate =  3088758.100647022

Erros
1.54735